In [1]:
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1574797416555_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
input_bucket = 's3://topic-sentiment-1'
input_path = '/data/*/*.json'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = spark.read.json(input_bucket + input_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------+--------------------+-------------+--------------------+--------------------+--------------------+---------+--------------------+
|             authors|      date_download|        date_modify|       date_publish|         description|            filename|           image_url|language|           localpath|source_domain|                text|               title|          title_page|title_rss|                 url|
+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------+--------------------+-------------+--------------------+--------------------+--------------------+---------+--------------------+
|[Edith Efron, Ron...|2019-11-26 18:18:07|2019-11-26 18:18:07|1994-11-01 05:00:00|The chaos and par...|1994_11_01__15747...|https://reason.co...|   

In [7]:
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
dir()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['F', 'HiveContext', 'SQLContext', 'SparkContext', 'SparkSession', 'StreamingContext', '__builtins__', 'cloudpickle', 'df', 'input_bucket', 'input_path', 'sc', 'spark', 'sqlContext']

In [11]:
SQLContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.context.SQLContext'>

In [12]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
run_sql('drop database if exists topic cascade')
run_sql('create database topic')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [18]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date_download: string (nullable = true)
 |-- date_modify: string (nullable = true)
 |-- date_publish: string (nullable = true)
 |-- description: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- language: string (nullable = true)
 |-- localpath: string (nullable = true)
 |-- source_domain: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- title_page: string (nullable = true)
 |-- title_rss: string (nullable = true)
 |-- url: string (nullable = true)

**TBD** Will I need to flatten the authors array?

In [19]:
permanent_table_name = 'topic.articles'
df.write.format("parquet").saveAsTable(permanent_table_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'pandas'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pandas'



In [25]:
result = run_sql('SELECT count(url) FROM topic.articles')
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|count(url)|
+----------+
|     41263|
+----------+

In [26]:
result = run_sql('SELECT source_domain, count(url) FROM topic.articles GROUP BY source_domain ORDER BY 2 DESC')
result.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|       source_domain|count(url)|
+--------------------+----------+
|       breitbart.com|      3750|
| fivethirtyeight.com|      3432|
|         nbcnews.com|      2270|
|              rt.com|      2068|
|           slate.com|      1878|
|         latimes.com|      1860|
|             bbc.com|      1852|
|     theatlantic.com|      1767|
|        theblaze.com|      1707|
|         foxnews.com|      1706|
|  chicagotribune.com|      1619|
|      denverpost.com|      1618|
|         nytimes.com|      1597|
|         thehill.com|      1338|
|          reason.com|      1281|
|         cbsnews.com|      1211|
|            time.com|      1206|
|           chron.com|      1170|
|       economist.com|       947|
|          forbes.com|       917|
|          boston.com|       912|
|           msnbc.com|       887|
|        dailykos.com|       668|
|             npr.org|       606|
|              ft.com|       596|
|          apnews.com|       561|
|washingtonexa

In [27]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum

def count_null(col_name):
    return spark_sum(col(col_name).isNull().cast('integer')).alias(col_name)

# Build up a list of column expressions, one per column.
exprs = [count_null(col_name) for col_name in df.columns]

# Run the aggregation. The *exprs converts the list of expressions into
# variable function arguments.
df.agg(*exprs).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+-----------+------------+-----------+--------+---------+--------+---------+-------------+----+-----+----------+---------+---+
|authors|date_download|date_modify|date_publish|description|filename|image_url|language|localpath|source_domain|text|title|title_page|title_rss|url|
+-------+-------------+-----------+------------+-----------+--------+---------+--------+---------+-------------+----+-----+----------+---------+---+
|      0|            0|          0|         618|       3826|       0|        0|    4963|        0|            0|1308|    3|         0|        0|  0|
+-------+-------------+-----------+------------+-----------+--------+---------+--------+---------+-------------+----+-----+----------+---------+---+

In [29]:
result = run_sql('SELECT count(url) FROM topic.articles WHERE text IS NULL')
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|count(url)|
+----------+
|      1308|
+----------+

In [30]:
result = run_sql("SELECT count(url) FROM topic.articles WHERE trim(text) = '' ")
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|count(url)|
+----------+
|         0|
+----------+

In [33]:
result = run_sql('SELECT date_publish, description, source_domain, title, title_page, url FROM topic.articles WHERE text IS NULL')
result.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [34]:
result = run_sql('SELECT source_domain, title, title_page, language FROM topic.articles WHERE language IS NULL')
result.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+--------------------+--------+
|source_domain|               title|          title_page|language|
+-------------+--------------------+--------------------+--------+
|    slate.com|The four big prob...|The four big prob...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    null|
|    slate.com|Help! I'm a snob ...|Help! I'm a snob ...|    n

In [35]:
result = run_sql('SELECT language, count(source_domain) FROM topic.articles GROUP BY language')
result.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+
|language|count(source_domain)|
+--------+--------------------+
|      en|               34896|
|      vi|                1397|
|    null|                4963|
|      ja|                   7|
+--------+--------------------+

**TBD** We will need to infer language when it is null, and then we want to filter to only include language = 'en'

**TBD** We also need to filter out articles with:
* date_publish IS NULL
* title IS NULL
* text IS NULL   (we might want to use description if text is missing)

In [42]:
result = run_sql('''
    SELECT date_format(date_publish, 'Y-M'), count(source_domain) 
    FROM topic.articles GROUP BY date_format(date_publish, 'Y-M')
    ORDER BY 1 DESC
    ''')
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------+--------------------+
|date_format(CAST(date_publish AS TIMESTAMP), Y-M)|count(source_domain)|
+-------------------------------------------------+--------------------+
|                                           2022-5|                   1|
|                                           2019-9|                 821|
|                                           2019-8|                 502|
|                                           2019-7|                 544|
|                                           2019-6|                 520|
|                                           2019-5|                 453|
|                                           2019-4|                 263|
|                                           2019-3|                  60|
|                                           2019-2|                 163|
|                                          2019-12|                   8|
|                                          2019-11|

**TBD** I may want to do some filtering on dates

In [44]:
result = run_sql('''
    SELECT count(url) FROM topic.articles WHERE text IS NULL and description IS NOT NULL
    ''')
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|count(url)|
+----------+
|      1228|
+----------+

This means only 90 articles are lacking description and text

In [45]:
result = run_sql('''
    SELECT title, source_domain, description FROM topic.articles WHERE text IS NULL and description IS NOT NULL
    ''')
result.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+--------------------+
|               title|     source_domain|         description|
+--------------------+------------------+--------------------+
|NBA: Rockets 101,...|         chron.com|HOUSTON - Kyle Lo...|
|Trump's Options f...|       nbcnews.com|The National Secu...|
|Wells Fargo Just ...|       nbcnews.com|Wells Fargo just ...|
|Susan Rice Speaks...|       nbcnews.com|Former National S...|
|Дабдабали анжуман...|           bbc.com|Қозоғистон сиёсат...|
|Ислом Каримов Ўзб...|           bbc.com|Президент Ислом К...|
|Mother Pleads for...|       nbcnews.com|Alyssa Ross and b...|
|Joaquin 'El Chapo...|       nbcnews.com|The drug kingpin ...|
|Эрон диний таълим...|           bbc.com|Тожикистон парлам...|
|México llama a G2...|         chron.com|México llama a G2...|
|Search Continues ...|       nbcnews.com|Alycia Yeoman dis...|
|Kushner Kin's Chi...|       nbcnews.com|The Kushner Compa...|
|More Details Rele...|       nbcnews.com|Terrence Wayne

In [54]:
clean_df = run_sql('''
    SELECT * FROM topic.articles 
    WHERE
      date_publish IS NOT NULL AND
      date_publish <= current_date() AND
      language = 'en' AND
      title IS NOT NULL AND
      (text IS NOT NULL OR description IS NOT NULL)
    ''').cache()
clean_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------+--------------------+-------------------+--------------------+--------------------+--------------------+---------+--------------------+
|             authors|      date_download|        date_modify|       date_publish|         description|            filename|           image_url|language|           localpath|      source_domain|                text|               title|          title_page|title_rss|                 url|
+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------+--------------------+-------------------+--------------------+--------------------+--------------------+---------+--------------------+
|[Edith Efron, Ron...|2019-11-26 18:18:07|2019-11-26 18:18:07|1994-11-01 05:00:00|The chaos and par...|1994_11_01__15747...|https:

In [55]:
from pyspark.sql.functions import expr

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
clean_df = clean_df.withColumn("text_or_desc", 
                         expr("case when text IS NULL then description ELSE text end"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
result = clean_df.groupBy('source_domain').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
result.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|       source_domain|count|
+--------------------+-----+
|              ft.com|  588|
|         nytimes.com| 1593|
|chicago.suntimes.com|   36|
|        theblaze.com| 1705|
|         reuters.com|    9|
|washingtonexamine...|  492|
|     theatlantic.com| 1758|
|         nbcnews.com| 1760|
|             wsj.com|   34|
|        usatoday.com|  208|
|           slate.com| 1708|
|         latimes.com| 1840|
|          boston.com|  894|
|       economist.com|  947|
|             bbc.com|  290|
|         mediate.com|    1|
|         foxnews.com| 1706|
|            time.com| 1183|
|          forbes.com|  660|
|         cbsnews.com| 1211|
|             npr.org|  599|
|     bostonglobe.com|   23|
|      denverpost.com| 1605|
|          reason.com| 1281|
|  chicagotribune.com| 1618|
|              rt.com|   96|
|          nypost.com|  227|
|     theguardian.com|  313|
|             vox.com|   33|
|       breitbart.com| 3745|
| fivethirtyeight.com| 3430|
+-------------

**TBD** Need to figure out why pandas didn't load